# 🏗️ YOLOv8 Segmentation Training - Kaggle

Train a YOLOv8 segmentation model to detect and segment:
- **Exposed Rebar** 🔩
- **Spalling** 🧱

---

## ⚡ Quick Start Guide

### Before You Begin:
1. **Enable GPU**: Settings → Accelerator → GPU T4 x2 (or P100)
2. **Enable Internet**: Settings → Internet → ON
3. **Get Roboflow API Key**: Go to https://app.roboflow.com/settings/api

### Run the Notebook:
- **Easy Mode**: Run → Run All (requires API key in Cell 6)
- **Step by Step**: Run each cell with Shift + Enter

---

## 📋 Requirements
- ✅ Kaggle with GPU enabled (T4 x2 or P100 recommended)
- ✅ Internet enabled (for dataset download)
- ✅ Roboflow API Key (free account - get it at https://app.roboflow.com/settings/api)
- ✅ ~2-4 hours training time

## 🎯 Training Pipeline
1. ✓ Check GPU availability
2. ✓ Install dependencies (Ultralytics + Roboflow)
3. ⚠️ **Download dataset** (YOU NEED API KEY HERE!)
4. ✓ Configure training parameters
5. ✓ Train YOLOv8 segmentation model
6. ✓ Visualize results & metrics
7. ✓ Test on sample images
8. ✓ Save trained model to Kaggle output


---
## 1️⃣ Check GPU Availability


In [ ]:
import torch
import os

# Check GPU
print("🔍 Checking GPU availability...\n")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU Device: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
    device = '0'  # Use first GPU
else:
    print("⚠️ No GPU detected! Training will be slow on CPU.")
    print("💡 Enable GPU: Settings → Accelerator → GPU T4 x2")
    device = 'cpu'

print(f"\n✅ Training device: {device}")
print(f"📂 Working directory: {os.getcwd()}")


---
## 2️⃣ Install Dependencies

This cell will:
1. Fix NumPy compatibility (downgrade from 2.x to 1.x if needed)
2. Install Ultralytics (YOLOv8)
3. Install Roboflow (for dataset download)

**Note:** Kaggle might have different package versions than Colab.


In [ ]:
print("📦 Installing dependencies...\n")

# Fix NumPy compatibility issue - AGGRESSIVE FIX
print("🔧 Step 1: Fixing NumPy compatibility...")
print("   Uninstalling NumPy 2.x...")
!pip uninstall numpy -y -q
print("   Installing NumPy 1.x...")
!pip install "numpy==1.26.4" -q
print("   ✅ NumPy 1.26.4 installed")

# Install ultralytics (YOLOv8)
print("\n🔧 Step 2: Installing Ultralytics (YOLOv8)...")
!pip install ultralytics -q

# Install roboflow
print("🔧 Step 3: Installing Roboflow...")
!pip install roboflow -q

print("\n✅ All dependencies installed successfully!")
print("✅ NumPy version fixed for compatibility")

# Verify installations
import numpy as np
print(f"\n📊 Installed versions:")
print(f"   NumPy: {np.__version__}")
try:
    from ultralytics import YOLO
    print(f"   Ultralytics: ✓ Installed")
except:
    print(f"   Ultralytics: ✗ Failed")
try:
    from roboflow import Roboflow
    print(f"   Roboflow: ✓ Installed")
except:
    print(f"   Roboflow: ✗ Failed")


---
## 3️⃣ Download Dataset from Roboflow

### ⚠️ IMPORTANT: Get Your Roboflow API Key

**Before running this cell, you MUST:**

1. Go to: https://app.roboflow.com/settings/api
2. Log in or create a free account
3. Copy your **Private API Key**
4. Paste it in the cell below, replacing `"YOUR_API_KEY_HERE"`

**Example:**
```python
ROBOFLOW_API_KEY = "abc123XYZ456"  # ← Replace with YOUR actual key
```

⚠️ **Do NOT run this cell until you replace the API key!**

⚠️ **Make sure Internet is enabled**: Settings → Internet → ON


In [ ]:
from roboflow import Roboflow

# 🔑 YOUR ROBOFLOW API KEY
# Get it from: https://app.roboflow.com/settings/api
ROBOFLOW_API_KEY = "orloumjlWtpPXoxK5bFa"  # ✅ API key configured!

# Validate API key
if ROBOFLOW_API_KEY == "YOUR_API_KEY_HERE":
    print("❌ ERROR: You need to replace 'YOUR_API_KEY_HERE' with your actual Roboflow API key!")
    print("\n📋 Steps to get your API key:")
    print("   1. Go to: https://app.roboflow.com/settings/api")
    print("   2. Log in to your Roboflow account (or create one - it's free!)")
    print("   3. Copy your 'Private API Key'")
    print("   4. Paste it above, replacing 'YOUR_API_KEY_HERE'")
    print("   5. Re-run this cell")
    print("\n💡 Example: ROBOFLOW_API_KEY = \"abc123XYZ456def789\"")
    raise ValueError("API key not configured")

# Initialize Roboflow
print("🔑 Initializing Roboflow with your API key...")
rf = Roboflow(api_key=ROBOFLOW_API_KEY)

# Download the Spalling and Exposed Rebar dataset
print("📦 Downloading dataset from Roboflow...")
print("   Workspace: labelling-9tvkx")
print("   Project: spalling-and-exposed-rebar-ttsjj")
print("   Version: 1")
print("\n⏳ This may take a few minutes...")

project = rf.workspace("labelling-9tvkx").project("spalling-and-exposed-rebar-ttsjj")
dataset = project.version(1).download("yolov8")

print(f"\n✅ Dataset downloaded successfully!")
print(f"📂 Location: {dataset.location}")
dataset_path = dataset.location

# Show dataset structure
import os
print(f"\n📊 Dataset Structure:")
for folder in ['train', 'valid', 'test']:
    folder_path = os.path.join(dataset.location, folder, 'images')
    if os.path.exists(folder_path):
        count = len([f for f in os.listdir(folder_path) if f.endswith('.jpg')])
        print(f"   {folder:6s}: {count} images")


---
## 💡 Troubleshooting - Common Issues

### 🔥 **KERNEL DYING / CRASHING (Most Common!)**

#### ❌ Error: "Kernel is dying" or kernel restarts during training
**Cause:** Out of Memory (OOM) - GPU ran out of memory!

**Solution:** Reduce batch size in Cell 10/11:
1. Go to the training configuration cell
2. Change: `'batch': 8` → `'batch': 4`
3. If still crashing: `'batch': 4` → `'batch': 2`
4. Re-run from Cell 10 onwards

**Quick fix:**
```python
CONFIG['batch'] = 4  # Add this line before training
```

**Note:** Smaller batch = slower training but more stable

---

### 📋 Installation Issues (Cell 4)

#### ❌ Error: "numpy.core.multiarray failed to import" or "NumPy 2.x incompatibility"
**Solution:** This is already fixed in Cell 4!
- Re-run **Cell 4** to downgrade NumPy to version 1.x
- You should see "NumPy: 1.x.x" in the output (not 2.x.x)

---

### 📦 Dataset Download Issues (Cell 6)

#### ❌ Error: "This API key does not exist"
**Solution:** You didn't replace `YOUR_API_KEY_HERE`!
- Go back to Cell 6
- Replace with your real key from https://app.roboflow.com/settings/api
- Re-run Cell 6

#### ❌ Error: "No internet connection" or "Connection timeout"
**Solution:** Internet is not enabled!
- Go to: Settings → Internet → Turn ON
- Re-run Cell 6

---

### ✅ If Everything Works:
You should see:
- Cell 4: "NumPy: 1.x.x" and all packages installed ✓
- Cell 6: "✅ Dataset downloaded successfully!" with image counts
- Cell 9: GPU memory status with recommendations

👇 **Continue to the next cell if you see the success messages above!**


---
## 4️⃣ Check GPU Memory (Important!)

Before training, let's check available GPU memory to avoid crashes.


In [ ]:
import torch

if torch.cuda.is_available():
    print("🔍 GPU Memory Status:\n")
    for i in range(torch.cuda.device_count()):
        gpu_memory = torch.cuda.get_device_properties(i).total_memory / 1024**3
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
        print(f"   Total Memory: {gpu_memory:.2f} GB")
        
        # Clear any cached memory
        torch.cuda.empty_cache()
        allocated = torch.cuda.memory_allocated(i) / 1024**3
        reserved = torch.cuda.memory_reserved(i) / 1024**3
        print(f"   Allocated: {allocated:.2f} GB")
        print(f"   Reserved: {reserved:.2f} GB")
        print(f"   Free: {gpu_memory - reserved:.2f} GB\n")
    
    # Recommendations based on GPU memory
    if gpu_memory < 16:
        print("⚠️  GPU has less than 16GB memory")
        print("💡 Recommended batch size: 4-8")
        print("💡 If kernel crashes, use batch=4 in next cell\n")
    else:
        print("✅ GPU has sufficient memory")
        print("💡 You can use batch size 8-16\n")
else:
    print("⚠️ No GPU detected! Training will be very slow on CPU.")


---
## 5️⃣ Configure Training Parameters


In [ ]:
# MINIMAL TEST CONFIGURATION - Find what's breaking!
CONFIG = {
    'model': 'yolov8n-seg.pt',
    'data': f'{dataset_path}/data.yaml',
    'epochs': 2,                     # Just 2 epochs to test!
    'imgsz': 320,                    # Very small
    'batch': 1,                      # Single image
    'device': device,
    'project': '/kaggle/working/runs/segment',
    'name': 'minimal_test',
    'save': False,                   # Don't save anything
    'plots': False,
    'verbose': True,
    'cache': False,
    'workers': 0,                    # Critical: no multiprocessing
    'rect': False,
    'single_cls': False,
    # Disable ALL augmentations
    'hsv_h': 0.0,
    'hsv_s': 0.0,
    'hsv_v': 0.0,
    'degrees': 0.0,
    'translate': 0.0,
    'scale': 0.0,
    'shear': 0.0,
    'perspective': 0.0,
    'flipud': 0.0,
    'fliplr': 0.0,
    'mosaic': 0.0,
    'mixup': 0.0,
    'copy_paste': 0.0,
}

print("🧪 MINIMAL TEST CONFIGURATION")
print("=" * 80)
print("⚠️  Kernel keeps dying even with workers=0!")
print("🎯 Using ABSOLUTE MINIMUM settings to isolate the problem")
print("=" * 80)
for key, value in CONFIG.items():
    print(f"   {key:20s}: {value}")
print("=" * 80)
print("\n🔍 THIS TEST WILL:")
print("   1. Run for just 2 epochs (~5 minutes)")
print("   2. Use smallest possible settings")
print("   3. All augmentations disabled")
print("   4. If this fails, we'll try detection model instead")
print("\n⏰ Should complete in 5-10 minutes if it works")
print("=" * 80)


---
## 🔄 ALTERNATIVE: Try Detection Model (If Segmentation Keeps Failing)

If the above keeps crashing, uncomment this cell to try **object detection** instead of segmentation.
Detection uses less memory and is more stable.

```python
# ALTERNATIVE CONFIG - DETECTION MODEL (not segmentation)
# Uncomment this entire section if segmentation keeps failing!

# CONFIG = {
#     'model': 'yolov8n.pt',           # Detection model (NOT segmentation!)
#     'data': f'{dataset_path}/data.yaml',
#     'epochs': 10,
#     'imgsz': 640,
#     'batch': 16,                     # Can use larger batch for detection
#     'device': device,
#     'project': '/kaggle/working/runs/detect',  # Different output folder
#     'name': 'spalling_detection',
#     'plots': False,
#     'verbose': True,
#     'cache': False,
#     'workers': 0,
# }
# 
# print("🔄 TRYING DETECTION MODEL INSTEAD")
# print("   Note: This will detect objects but NOT segment them")
# print("   (Bounding boxes only, no masks)")
```


---
## 6️⃣ Clear GPU Memory (IMPORTANT!)

Before training, let's aggressively clear any cached GPU memory.


In [ ]:
import torch
import gc

# Clear Python garbage
gc.collect()

# Clear GPU cache
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    
    # Check memory after clearing
    allocated = torch.cuda.memory_allocated(0) / 1024**3
    reserved = torch.cuda.memory_reserved(0) / 1024**3
    
    print("🧹 GPU Memory Cleared!")
    print(f"   Allocated: {allocated:.2f} GB")
    print(f"   Reserved: {reserved:.2f} GB")
    print("\n✅ GPU is ready for training!")
else:
    print("⚠️ No GPU available - training on CPU will be very slow")


---
## 7️⃣ Initialize Model and Start Training

⏰ **Training time: 3-4 hours** (with batch=8, imgsz=640 - Normal speed!)

**🎯 THE FIX: workers=0**
- Kernel was dying BEFORE first epoch (not during training)
- This means it was a **data loading issue**, not memory!
- Setting `workers=0` disables problematic multiprocessing

**GPU Memory Monitor (with batch=8):**
- ✅ **8-10 GB**: Normal (plenty of headroom on T4's 15GB)
- ⚠️ **> 13 GB**: Approaching limit
- ❌ **> 14 GB**: Will crash

**What Changed:**
- ✅ **Batch size: 8** (not 1!) - Normal training
- ✅ **Image size: 640** (not 384!) - Full quality
- ✅ **Workers: 0** - This was the actual issue!
- ✅ **3-4 hours** instead of 10-12 hours!


In [ ]:
from ultralytics import YOLO
import time
import traceback
import sys

print("\n" + "🚀" * 40)
print("STARTING YOLOV8 SEGMENTATION TRAINING - KAGGLE")
print("🚀" * 40 + "\n")

try:
    # Initialize model
    print("🔧 Step 1: Loading YOLOv8n-seg pretrained model...")
    model = YOLO(CONFIG['model'])
    print("✅ Model loaded successfully!\n")
    
    # Verify dataset
    print("🔧 Step 2: Verifying dataset...")
    print(f"   Dataset path: {CONFIG['data']}")
    import os
    if os.path.exists(CONFIG['data']):
        print("✅ Dataset file exists!\n")
    else:
        raise FileNotFoundError(f"Dataset not found: {CONFIG['data']}")
    
    # Start training
    print("🏋️  Step 3: Starting training...\n")
    print("=" * 80)
    print(f"📂 Results will be saved to: {CONFIG['project']}/{CONFIG['name']}")
    print(f"💾 Model checkpoints: /kaggle/working/ (saved as output)")
    print("=" * 80 + "\n")
    
    print("⏰ Training started at:", time.strftime("%Y-%m-%d %H:%M:%S"))
    print("\n📊 Watch the GPU_mem column below:")
    print("   Should stay around 2-4 GB with batch=1")
    print("   If it goes > 10 GB, it might crash\n")
    print("=" * 80 + "\n")
    
    start_time = time.time()
    
    # Train with error catching
    results = model.train(**CONFIG)
    
    training_time = time.time() - start_time
    
    print("\n" + "=" * 80)
    print("🎉 TRAINING COMPLETED SUCCESSFULLY! 🎉")
    print("=" * 80)
    print(f"\n⏱️  Total training time: {training_time/3600:.2f} hours")
    print(f"📂 Model saved to: {CONFIG['project']}/{CONFIG['name']}/weights/best.pt")
    print(f"💾 All results saved to: /kaggle/working/runs/segment/{CONFIG['name']}/")
    print("\n💡 Files in /kaggle/working/ are automatically saved as notebook output!")
    
except KeyboardInterrupt:
    print("\n⚠️  Training interrupted by user (Ctrl+C)")
    print("💾 Partial results may be saved in:", CONFIG['project'])
    
except RuntimeError as e:
    print("\n" + "=" * 80)
    print("❌ RUNTIME ERROR DETECTED!")
    print("=" * 80)
    error_msg = str(e)
    print(f"\nError message: {error_msg}\n")
    
    # Check if it's OOM
    if "out of memory" in error_msg.lower() or "oom" in error_msg.lower():
        print("🔥 This IS a GPU Out of Memory error!")
        print("\n💡 Solutions:")
        print("   1. Reduce batch size further (already at minimum!)")
        print("   2. Reduce image size: CONFIG['imgsz'] = 320")
        print("   3. Try detection instead: 'yolov8n.pt'")
    else:
        print("🤔 This is NOT an OOM error - it's something else!")
        print("\n💡 Possible causes:")
        print("   - Dataset loading issue")
        print("   - Corrupted images")
        print("   - PyTorch/CUDA compatibility")
        print("   - Augmentation bug")
    
    print(f"\nFull traceback:")
    traceback.print_exc()
    
except Exception as e:
    print("\n" + "=" * 80)
    print("❌ UNEXPECTED ERROR!")
    print("=" * 80)
    print(f"\nError type: {type(e).__name__}")
    print(f"Error message: {str(e)}\n")
    print(f"Full traceback:")
    traceback.print_exc()
    
    print("\n💡 This error information will help debug the issue!")
    print("   Please share this error message for help.")
    
finally:
    # Cleanup
    import torch
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        print("\n🧹 GPU cache cleared")


---
## 6️⃣ Visualize Training Results


In [ ]:
from IPython.display import Image, display
import glob

results_dir = f"{CONFIG['project']}/{CONFIG['name']}"

print("📈 Training Results Visualizations:\n")

# Display results plot
if os.path.exists(f"{results_dir}/results.png"):
    print("📊 Training Curves:")
    display(Image(filename=f"{results_dir}/results.png"))

# Display confusion matrix
if os.path.exists(f"{results_dir}/confusion_matrix_normalized.png"):
    print("\n🎯 Confusion Matrix:")
    display(Image(filename=f"{results_dir}/confusion_matrix_normalized.png"))

# Display validation predictions
val_images = glob.glob(f"{results_dir}/val_batch*_pred.jpg")
if val_images:
    print("\n🔍 Validation Predictions:")
    for img_path in val_images[:3]:  # Show first 3 batches
        display(Image(filename=img_path, width=800))


---
## 7️⃣ Evaluate Model Performance


In [ ]:
import pandas as pd

# Load results CSV
results_csv = f"{results_dir}/results.csv"
if os.path.exists(results_csv):
    df = pd.read_csv(results_csv)
    df.columns = df.columns.str.strip()
    
    print("📊 Final Training Metrics:\n")
    print("=" * 80)
    
    # Get last epoch metrics
    last_epoch = df.iloc[-1]
    
    metrics = [
        ('Box Precision', 'metrics/precision(B)'),
        ('Box Recall', 'metrics/recall(B)'),
        ('Box mAP50', 'metrics/mAP50(B)'),
        ('Box mAP50-95', 'metrics/mAP50-95(B)'),
        ('Mask Precision', 'metrics/precision(M)'),
        ('Mask Recall', 'metrics/recall(M)'),
        ('Mask mAP50', 'metrics/mAP50(M)'),
        ('Mask mAP50-95', 'metrics/mAP50-95(M)'),
    ]
    
    for name, col in metrics:
        if col in df.columns:
            print(f"{name:25s}: {last_epoch[col]:.4f}")
    
    print("=" * 80)
    
    # Show last 10 epochs
    print("\n📉 Last 10 Epochs Performance:")
    print(df[['epoch', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)', 
              'metrics/mAP50(M)', 'metrics/mAP50-95(M)']].tail(10).to_string(index=False))
else:
    print("⚠️ Results CSV not found")


---
## 8️⃣ Test Model on Sample Images


In [ ]:
# Load the best trained model
best_model_path = f"{results_dir}/weights/best.pt"
model = YOLO(best_model_path)

# Get some test images
test_images = glob.glob(f"{dataset_path}/test/images/*.jpg")[:5]

if test_images:
    print("🔬 Running inference on test images...\n")
    
    for img_path in test_images:
        # Run inference
        results = model(img_path)
        
        # Plot results
        for r in results:
            im_array = r.plot()
            
            # Display
            from PIL import Image as PILImage
            import matplotlib.pyplot as plt
            
            plt.figure(figsize=(12, 8))
            plt.imshow(im_array[..., ::-1])
            plt.axis('off')
            plt.title(f"Prediction: {os.path.basename(img_path)}")
            plt.tight_layout()
            plt.show()
            
            # Print detections
            if len(r.boxes) > 0:
                print(f"   Detected {len(r.boxes)} objects in {os.path.basename(img_path)}")
                for i, box in enumerate(r.boxes):
                    cls_id = int(box.cls[0])
                    conf = float(box.conf[0])
                    cls_name = model.names[cls_id]
                    print(f"      {i+1}. {cls_name}: {conf:.3f}")
            else:
                print(f"   No objects detected in {os.path.basename(img_path)}")
            print()
else:
    print("⚠️ No test images found")


In [ ]:
import shutil

# The model is already in /kaggle/working/runs/ so it will be saved
# Let's also copy the best model to the working directory root for easy access
best_model = f"{results_dir}/weights/best.pt"
last_model = f"{results_dir}/weights/last.pt"

if os.path.exists(best_model):
    shutil.copy(best_model, "/kaggle/working/best.pt")
    print(f"✅ Copied best model to: /kaggle/working/best.pt")
    print(f"   File size: {os.path.getsize('/kaggle/working/best.pt') / 1024**2:.2f} MB")

if os.path.exists(last_model):
    shutil.copy(last_model, "/kaggle/working/last.pt")
    print(f"✅ Copied last model to: /kaggle/working/last.pt")
    print(f"   File size: {os.path.getsize('/kaggle/working/last.pt') / 1024**2:.2f} MB")

# Create a results summary
print(f"\n📂 All Training Results:")
print(f"   Model weights: {results_dir}/weights/")
print(f"   Training curves: {results_dir}/results.png")
print(f"   Confusion matrix: {results_dir}/confusion_matrix_normalized.png")
print(f"   Results CSV: {results_dir}/results.csv")
print(f"\n💾 To download:")
print(f"   1. Click 'Output' tab at the top")
print(f"   2. Find your files in the output section")
print(f"   3. Click download button")
print(f"\n📁 Working directory contents:")
!ls -lh /kaggle/working/*.pt 2>/dev/null || echo "   (model files listed above)"


---
## 🔟 Export Model (Optional)

Export to ONNX format for deployment


In [ ]:
# Export to ONNX format (for broader deployment)
print("🔄 Exporting model to ONNX format...\n")

model = YOLO(best_model_path)
export_path = model.export(format='onnx')

# Copy to working directory
if os.path.exists(export_path):
    shutil.copy(export_path, "/kaggle/working/best.onnx")
    print(f"\n✅ Model exported to: /kaggle/working/best.onnx")
    print(f"   File size: {os.path.getsize('/kaggle/working/best.onnx') / 1024**2:.2f} MB")

print("\n📦 Available export formats:")
print("   - PyTorch (.pt) ✓")
print("   - ONNX (.onnx) ✓")
print("   - TensorRT (.engine)")
print("   - CoreML (.mlmodel)")
print("   - TFLite (.tflite)")
print("\n💡 To export to other formats, use: model.export(format='<format>')")


---
## 📝 Summary

### Training Complete! 🎉

Your YOLOv8 segmentation model has been trained to detect and segment:
- **Exposed Rebar** 🔩
- **Spalling** 🧱

### 📂 Output Files (in Kaggle Output):
- `best.pt` - Best model weights (lowest validation loss)
- `last.pt` - Last epoch checkpoint
- `best.onnx` - ONNX export (optional)
- `runs/segment/spalling_rebar_training/` - All training results

### 💾 How to Download from Kaggle:
1. Click the **"Output"** tab at the top of the page
2. Find your files in the output section
3. Click the **download** button next to each file
4. Or download the entire output as a zip

### 🚀 Next Steps:
1. Download the `best.pt` model
2. Use it for inference on new images
3. Deploy in your application
4. Fine-tune with more data if needed

### 💻 Usage Example:
```python
from ultralytics import YOLO

# Load model
model = YOLO('best.pt')

# Run inference
results = model('path/to/image.jpg')

# Display results
results[0].show()

# Get segmentation masks
masks = results[0].masks  # Segmentation masks
boxes = results[0].boxes  # Bounding boxes
```

### 📊 Expected Performance:
- **Box mAP50**: ~75-85%
- **Mask mAP50**: ~70-80%
- **Training time**: 2-4 hours on T4 GPU
- **Model size**: ~6MB (YOLOv8n-seg)

---

**Happy Segmenting! 🎯**

**All your files are saved in the Output tab. Check the "Output" section above to download your trained model!**
